## Q1. Running Elastic 

![running_elastic](../images/01_q1.png)

The answer is: **42f05b9372a9a4a470db3b52817899b99a76ee73**

## Getting the data

In [3]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

## Q2. Indexing the data

In [4]:
from elasticsearch import Elasticsearch

In [5]:
es_client = Elasticsearch("http://localhost:9200")

In [6]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions"

In [7]:
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [8]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:25<00:00, 37.47it/s]


As we can see above, the function to insert the documents is **index**

## Q3. Searching

In [10]:
query = "How do I execute a command in a running docker container?"

In [11]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
        }
    }
}

In [12]:
response = es_client.search(index=index_name, body=search_query)

In [13]:
result_hits = []

for hit in response["hits"]["hits"]:
    result_hits.append(hit["_score"])

In [14]:
result_hits

[84.050095, 75.54128, 72.08518, 51.04628, 49.938507]

The best score is: **84.05**

## Q4. Filtering

In [15]:
search_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

In [16]:
response = es_client.search(index=index_name, body=search_query)

In [17]:
result_docs = []

for hit in response["hits"]["hits"]:
    result_docs.append(hit["_source"]["question"])

In [18]:
result_docs

['How do I debug a docker container?',
 'How do I copy files from my local machine to docker container?',
 'How do I copy files from a different folder into docker container’s working directory?']

The third question returned is:

**"How do I copy files from a different folder into docker container’s working directory?"**

## Q5. Building a prompt

In [19]:
result_docs = []

for hit in response["hits"]["hits"]:
    result_docs.append(hit["_source"])

In [20]:
context_template = """
Q: {question}
A: {text}
""".strip()

In [49]:
context = ""
for i, doc in enumerate(result_docs):
    context = context + context_template.format(question=doc['question'], text=doc['text'])
    if (i != len(result_docs) - 1):
        context = context + "\n\n"

In [50]:
query = "How do I execute a command in a running docker container?"

In [51]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [52]:
prompt = prompt_template.format(question=query, context=context)

In [54]:
len(prompt)

1462

The resulting prompt has a length of: **1462**

## Q6. Tokens

In [55]:
import tiktoken

In [56]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [57]:
encoded_prompt = encoding.encode(prompt)

In [58]:
len(encoded_prompt)

322

The encoded prompt has **322** tokens